In [18]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer
linkage_name = '20190603_131951_meshID_96824868-7c0d-47f9-a858-1cdeb1edb0a7'
l = elastic_rods.RodLinkage('../../examples/florin/{}.obj'.format(linkage_name), 8)
#driver=166
driver=l.centralJoint()

#mat = elastic_rods.RodMaterial('+', 2000, 0.3, [5, 5, 0.7, 0.7], stiffAxis=elastic_rods.StiffAxis.D1)
mat = elastic_rods.RodMaterial('rectangle', 20000, 0.3, [12, 8])
l.setMaterial(mat)

with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024)
#view.setCameraParams(((2.656451698624758, 1.5087349593639046, 0.23168476156681694),
# (0.052583904614345374, 0.026224725204501566, 0.9982721055721517),
# (0.13680349418553736, -0.13680349418553736, -0.8425278443781967)))
view.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [5]:
l.saveVisualizationGeometry('deployed_{}.obj'.format(linkage_name))
#l.writeLinkageDebugData('')

In [17]:
from write_render_files import writeRenderFiles, writeActuators
writeRenderFiles(l, 'Pavilion21', 'pav21')
#writeActuators(l, np.where(torques > 1e-4)[0], directory='AsymPointyDataFlat', name='asymPointy_Fab')

In [10]:
from linkage_utils import writeRodSegments
writeRodSegments(l,'rod_segments_{}.txt'.format(linkage_name), zeroBasedIndexing=True)
np.savetxt('restlen_{}.txt'.format(linkage_name),l.getPerSegmentRestLength())
np.savetxt('normals_{}.txt'.format(linkage_name), np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_{}.msh'.format(linkage_name))

In [41]:
# Compute maximum "stress" appearing at any rod vertex in the structure
max([max(segmentStress) for segmentStress in l.rodStresses()[1]])

24.37145054902055

In [ ]:
# Output actuation torque
for forceScale in np.linspace(1.0, 0, 51):
    name = 'Pavilion14_{}'.format(forceScale)
    with suppress_stdout(): elastic_rods.compute_equilibrium(l, forceScale * externalForces, fixedVars=fixedVars)
    view.update(preserveExisting=True)
    writeRenderFiles(l, directory, name)
    writeActuators(l, np.where(torques > 0.1)[0], directory, name)

In [44]:
# Compute average "stress" appearing over all rod vertices in the structure
np.mean(sum(l.rodStresses()[1], []))

11.81760497978405

In [21]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
# open_linkage(l, driver, 2 * np.pi/3, 25, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=False);
open_linkage(l, driver, np.deg2rad(85) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);

target angle:  1.654572130890621
0	259996	546.383	546.383	1	0
Backtracking failed.
1.5425373719533757	259985.38718188612	259985.38718188618	256844.50796709795	3078.3220826725687	62.55713211577429
target angle:  1.6510814723866323
0	258429	543.504	543.504	1	0
1.5385122960367887	258418.24456343896	258418.24456343893	255288.8851556733	3067.447106309446	61.91230145613677
target angle:  1.6475908138826436
0	256873	540.659	540.659	1	0
Backtracking failed.
1.5344984835758677	256862.73591746608	256862.73591746605	253744.48812058958	3056.9731149455547	61.27468193096383
target angle:  1.644100155378655
0	255329	537.794	537.794	1	0
1.530495914792947	255318.77499664377	255318.77499664368	252211.23331698397	3046.8974197686916	60.64425989098388
target angle:  1.6406094968746663
0	253797	534.963	534.963	1	0
1.52650456854767	253786.27553187124	253786.27553187124	250689.03704444645	3037.2174796603836	60.021007764329426
target angle:  1.6371188383706776
0	252276	532.124	532.124	1	0
1.5225244218619776	25